In [4]:
# import fastf1
# import pandas as pd
# from pymongo import MongoClient
# import os
# import time
# import random
#
# # 1. MongoDB 연결
# client = MongoClient("mongodb://localhost:27017/")
# db = client["f1_bigdata_db"]
# col_telemetry = db["telemetry_raw"]
# col_races = db["race_meta"]
#
# def reset_database():
#     """
#     기존에 수집된 데이터가 있다면 모두 삭제하여 초기화합니다.
#     (타이어 데이터가 없는 구버전 데이터를 없애기 위함)
#     """
#     print("🔥 [시스템 초기화] 기존 DB 데이터를 삭제하고 새로 준비합니다...")
#     col_telemetry.drop()
#     col_races.drop()
#     print("✅ 초기화 완료. 수집을 시작합니다.\n")
#
# reset_database()

🔥 [시스템 초기화] 기존 DB 데이터를 삭제하고 새로 준비합니다...
✅ 초기화 완료. 수집을 시작합니다.



In [1]:
import fastf1
import pandas as pd
from pymongo import MongoClient
import os
import time
import random

# 1. MongoDB 연결
client = MongoClient("mongodb://localhost:27017/")
db = client["f1_bigdata_db"]
col_telemetry = db["telemetry_raw"]
col_races = db["race_meta"]

# 2. 캐시 설정
cache_dir = "D:/f1_cache_temp"
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)
fastf1.Cache.enable_cache(cache_dir)

def collect_f1_data_smart(start_year=2018, end_year=2024):
    print(f"🚀 [스마트 수집] F1 데이터 수집 시작 (이어받기 모드): {start_year} ~ {end_year}")

    for year in range(start_year, end_year + 1):
        try:
            print(f"\n📅 {year}년도 스케줄 로딩 중...")
            schedule = fastf1.get_event_schedule(year)
            races = schedule[schedule['EventFormat'] != 'testing']

            if races.empty:
                print(f"❌ {year}년 스케줄 없음 (Skip)")
                continue

            for _, race in races.iterrows():
                round_num = race['RoundNumber']
                event_name = race['EventName']

                # [핵심] 이미 DB에 데이터가 충분히(예: 1만 건 이상) 있다면 건너뜁니다!
                # 이렇게 해야 중복 요청을 안 보내서 차단을 피합니다.
                if col_telemetry.count_documents({"Year": year, "Round": round_num}) > 10000:
                    print(f"  ⏭️ [{year} R{round_num}] {event_name} - 이미 수집 완료 (Skip)")
                    continue

                print(f"  📥 [{year} R{round_num}] {event_name} 수집 중...", end="")

                try:
                    session = fastf1.get_session(year, round_num, 'R')
                    session.load(telemetry=True, weather=True, messages=False)

                    # 1. 경기 메타 정보 (Upsert: 있으면 갱신, 없으면 추가)
                    race_meta = {
                        "year": year, "round": round_num, "event_name": event_name,
                        "circuit": session.event.Location,
                        "date": str(session.event.EventDate)
                    }
                    col_races.update_one(
                        {"year": year, "round": round_num},
                        {"$set": race_meta},
                        upsert=True
                    )

                    # 2. 텔레메트리 + 타이어 데이터 저장
                    # (혹시 모를 중복 방지를 위해 해당 라운드 데이터 삭제 후 삽입)
                    col_telemetry.delete_many({"Year": year, "Round": round_num})

                    driver_count = 0
                    for driver in session.drivers:
                        try:
                            laps = session.laps.pick_drivers(driver)
                            if laps.empty: continue

                            all_lap_data = []
                            for _, lap in laps.iterlaps():
                                car_data = lap.get_car_data().add_distance()
                                car_data['Driver'] = driver
                                car_data['Year'] = year
                                car_data['Round'] = round_num
                                car_data['LapNumber'] = lap['LapNumber']
                                car_data['Compound'] = lap['Compound']
                                car_data['TyreLife'] = lap['TyreLife']
                                all_lap_data.append(car_data)

                            if all_lap_data:
                                full_telemetry = pd.concat(all_lap_data)
                                records = full_telemetry.to_dict('records')
                                clean_records = []
                                for r in records:
                                    if 'Time' in r: r['Time'] = r['Time'].total_seconds()
                                    if 'SessionTime' in r: r['SessionTime'] = r['SessionTime'].total_seconds()
                                    if 'Date' in r: r['Date'] = str(r['Date'])
                                    if pd.isna(r.get('Compound')): r['Compound'] = "UNKNOWN"
                                    if pd.isna(r.get('TyreLife')): r['TyreLife'] = 0
                                    clean_records.append(r)

                                if clean_records:
                                    col_telemetry.insert_many(clean_records)
                                    driver_count += 1
                        except:
                            continue

                    print(f" ✅ 완료 (드라이버 {driver_count}명)")

                    # [차단 방지] 한 경기 끝날 때마다 5~10초 휴식
                    time.sleep(random.randint(5, 10))

                except Exception as e:
                    print(f" ❌ 실패 ({e})")
                    time.sleep(30)

        except Exception as e:
            print(f"❌ {year}년 스케줄 에러: {e}")
            time.sleep(60)

# --- 실행 ---
collect_f1_data_smart(2023, 2024)

🚀 [스마트 수집] F1 데이터 수집 시작 (이어받기 모드): 2023 ~ 2024

📅 2023년도 스케줄 로딩 중...


core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


  📥 [2023 R1] Bahrain Grand Prix 수집 중...

req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written to cache!
core           INFO 	Processing timing data...
req            INFO 	No cached data found for car_data. Loading data...
_api           INFO 	Fetching car data...
_api           INFO 	Parsing car data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data 

 ✅ 완료 (드라이버 20명)


...

 ✅ 완료 (드라이버 20명)


core           INFO 	Loading data for Abu Dhabi Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


  📥 [2024 R24] Abu Dhabi Grand Prix 수집 중...

req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
core           INFO 	Finished loading data for 20 drivers: ['4', '55', '16', '44', '63', '1', '10', '27', '14', '81', '23', '22', '24', '18', '61', '20', '30', '77', '43', '11']


 ✅ 완료 (드라이버 20명)
